# Graph analysis

Your task in this exercise is straightforward: produce a graph of the named entities in the literary corpus and calculate a handful of centrality measures on it. This follows notebook 09 very closely.

Your notebook should:

1. Perform entity recognition on the full corpus using Spacy.
2. Produce a `networkx` graph object of the extracted entites, where nodes are entities and edges represent weighted coöccurrence in the same text. (For a challenge, use the chunked version of the corpus to get better in-document coöccurrence resolution.) **NB.** You can use all the Spacy entity types if you want, but this produces a very large graph, which in turn makes some of the centrality measures very computationaly expensive. I'd suggest limiting the retained entity types to just a few (ideally, broadly related, like 'GPE', 'LOC', and 'FACILITY', for example).
3. Produce a list of the top 20 entities in your graph according to each of at least three different centrality measures.
4. Produce a visualization of your graph. In a perfect world, make this visualization as legible as possible ;).

There's starter code from the textbook below. Modify it as needed and add your own code to perform the tasks listed above. Upload your completed notebook to Sakai.

In [1]:
import spacy
import heapq
import itertools
import networkx as nx
import os
import sys
import time

from operator import itemgetter

# Import our libraries
sys.path.append(os.path.join('..', 'libraries'))
from TMN import PickledCorpusReader

nlp = spacy.load('en')

GOOD_ENTS = ['PERSON', 'NORP', 'FACILITY', 'ORG', 'GPE', 'LOC',
             'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LANGUAGE']

def entities(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        #  filter out non-target entities
        if ent.label_ in GOOD_ENTS:
            return ent.text, ent.label_
        else:
            pass

def pairs(doc):
    candidates = [
        entities(' '.join(word for word, tag in sent))
        for para in doc for sent in para
    ]

    doc_entities = [
        entity for entity in candidates if entity is not None
    ]

    return list(itertools.permutations(set(doc_entities), 2))

def graph(docs):
    G = nx.Graph()
    completed = 0
    for doc in docs:
        if completed % 500 == 0:
            print(f'{time.ctime()} :: {completed} chunks complete')
        for pair in pairs(doc):
            if (pair[0][0], pair[1][0]) in G.edges():
                G.edges[(pair[0][0], pair[1][0])]['weight'] += 1
            else:
                G.add_edge(pair[0][0], pair[1][0], weight=1)
        completed+=1
    return G

def nbest_centrality(G, metric, n=10, attr="centrality", **kwargs):
    # Compute the centrality scores for each vertex
    scores = metric(G, **kwargs)

    # Set the score as a property on each node
    nx.set_node_attributes(G, name=attr, values=scores)

    # Find the top n scores and print them along with their index
    topn = heapq.nlargest(n, scores.items(), key=itemgetter(1))
    for idx, item in enumerate(topn):
        print("{}. {}: {:0.4f}".format(idx + 1, *item))

    return G